In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('drive/MyDrive/DL final project') # add shortcut to the DL final project to your google drive

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# Check that files are good

In [4]:
counter = 0
good_idx = []

for i in range(160):
  gaze_path = 'raw_data/all gaze/User '+str(i)+'_all_gaze.csv'
  os_path = 'raw_data/our scale OS files/subject-'+str(i)+'.csv'

  try:
    if i == 0:
      gaze = pd.read_csv(gaze_path, delimiter=',')
    else:
      gaze = pd.read_csv(gaze_path)

    if os.path.isfile(os_path):

      info = pd.read_csv(os_path, sep=None, engine='python') #, delimiter=';')

      gaze_time_col = [i for i in gaze.columns.values if i[:5] == 'TIME('][0]
      gaze_start = datetime.strptime(gaze_time_col[5:-1], '%Y/%m/%d %H:%M:%S.%f')
      img_start = datetime.strptime(info.start_time_system[0], '%d/%m/%y %H:%M:%S.%f')

      if abs((img_start.timestamp() - gaze_start.timestamp()) * 1000) > 1000000:
        print(gaze_path, gaze_start)
        print(os_path, img_start)
        print("FILES HAVE BIG TIME DIFFERENCE")
        counter += 1
        print()
      else:
        good_idx.append(i)

    else:
      print(os_path, 'NO OS FILE')
      counter += 1
      print()

  except FileNotFoundError:
    print(gaze_path, 'NO GAZE FILE')
    counter += 1
    print()

# Cut files into "epochs"

In [107]:
for i in range(160):

  if not os.path.exists('prep_data/epochs/'+str(i)):
    os.mkdir('prep_data/epochs/'+str(i))

  gaze_path = 'raw_data/all gaze/User '+str(i)+'_all_gaze.csv'
  os_path = 'raw_data/our scale OS files/subject-'+str(i)+'.csv'

  if i == 0:
    gaze = pd.read_csv(gaze_path, delimiter=',')
  else:
    gaze = pd.read_csv(gaze_path)

  try:
    info = pd.read_csv(os_path, delimiter=',')
    img_start = datetime.strptime(info.start_time_system[0], '%d/%m/%y %H:%M:%S.%f')
  except:
    info = pd.read_csv(os_path, delimiter=';')
    img_start = datetime.strptime(info.start_time_system[0], '%d/%m/%y %H:%M:%S.%f')

  gaze_time_col = [i for i in gaze.columns.values if i[:5] == 'TIME('][0]
  gaze_start = datetime.strptime(gaze_time_col[5:-1], '%Y/%m/%d %H:%M:%S.%f')

  gaze = gaze[[gaze_time_col, 'FPOGX', 'FPOGY', 'FPOGID', 'FPOGV', 'FPOGV', 'LPMM', 'LPMMV', 'RPMM', 'RPMMV', 'GSR', 'GSRV', 'HR', 'HRV']]
  info = info[['Image', 'start_time_system', 'time_Images', 'form_response']]

  counter = 0

  for img_start_time, image, arousal, valence in zip(
      ((img_start - gaze_start).total_seconds() + np.array(info.time_Images)[1::2] / 1000),
      np.array(info.Image)[1::2],
      np.array(info.form_response)[1::2],
      np.array(info.form_response)[2::2]):
    
    idx = (np.where((gaze[gaze_time_col] - img_start_time)>0)[0][0])
    epoch_gaze = gaze.loc[idx:(idx+300)].copy()
    epoch_gaze['Image'] = image
    epoch_gaze['Arousal'] = arousal
    epoch_gaze['Valence'] = valence

    epoch_gaze.to_csv('prep_data/epochs/'+str(i)+'/'+str(counter)+'_'+image.split('/')[-1].split('.')[0]+'_'+str(int(arousal))+'_'+str(int(valence))+'.csv')
    counter += 1

KeyboardInterrupt: ignored

In [109]:
!unzip 'prep_data.zip'

unzip:  cannot find or open prep_data.zip, prep_data.zip.zip or prep_data.zip.ZIP.


In [110]:
import zipfile

In [114]:
zip_ref = zipfile.ZipFile("prep_data.zip", 'r')
zip_ref.extractall("")
zip_ref.close()

FileNotFoundError: ignored

In [4]:
os.listdir()

['final_emotion.ipynb', 'multimodal_nn.jpg', 'preprocessing.ipynb', 'raw_data']